In [ ]:
import pandas as pd
import numpy as np

# Silence some warnings about deprecated Pandas behavior
pd.set_option("future.no_silent_downcasting", True)

In [ ]:
# Utility functions
def melt_monthly_vars(pr_gen_fuel: pd.DataFrame, variable_name: str) -> pd.DataFrame:
    """Melt many columns of monthly data for a single variable into a month column and a value column.

    This code takes a table with data stored in one column per month and stacks all the fields for a single variable (fuel_consumed_for_electricity_mmbtu), returning a table with one month column and one value column for this variable in
    order to make it easier to plot our data over time. Note that this drops the other variables of data.

    Args:
        pr_gen_fuel: EIA 923 Puerto Rico generation fuel data.
        variable_name: The variable to be melted.

    Returns:
        var_melt: A dataframe containing only index columns, a month column and the melted variable data.
    """
    var_cols = index_cols + [col for col in pr_gen_fuel.columns if col.startswith(variable_name)]
    var_df = pr_gen_fuel.loc[:, var_cols]

    ## Melt the fuel_consumed columns
    var_melt = var_df.melt(
        id_vars=index_cols,
        var_name="month",
        value_name=variable_name
    )
    var_melt["month"] = var_melt["month"].str.replace(f"{variable_name}_", "")
    var_melt = var_melt.set_index(index_cols + ["month"])
    return var_melt

def handle_data_types(pr_df: pd.DataFrame, categorical_cols: list[str]) -> pd.DataFrame:
    """Convert EIA 923 PR columns into desired data types.

    In addition to using the standard convert_dtypes() function, handle a series of
    non-standard data types conversions for associated_combined_heat_power
    and create categorical columns to save memory.

    Args:
        pr_df: Dataframe with EIA 923 Puerto Rico data.
        categorical_cols: List of columns that should be converted to a categorical dtype.

    Returns:
        pr_df: A dataframe with converted data types.
    """
    pr_df = pr_df.convert_dtypes()
    pr_df["associated_combined_heat_power"] = (
        pr_df["associated_combined_heat_power"]
        .astype("object") # necessary for the types to work for the .replace() call
        .replace({"Y": True, "N": False})
        .astype("boolean")
    )
    pr_df = pr_df.astype({col: "category" for col in categorical_cols})
    return pr_df

In [ ]:
# Read in the raw data
pr_gen_fuel = pd.read_parquet("../data/raw_eia923__puerto_rico_generation_fuel.parquet")
pr_plant_frame = pd.read_parquet("../data/raw_eia923__puerto_rico_plant_frame.parquet")

In [ ]:
# Handle EIA null values
pr_gen_fuel = pr_gen_fuel.replace(to_replace = ".", value = pd.NA)

# Convert data types (mmbtu/units to numeric, booleans, categories)
pr_gen_fuel = handle_data_types(
        pr_gen_fuel,
        categorical_cols = ["energy_source_code","fuel_type_code_agg", "prime_mover_code", "reporting_frequency_code", "data_maturity", "plant_state"]
                            )

for colname in pr_gen_fuel.columns: # TODO: Do we need this? Check.
    if (
        "fuel_consumption" in colname
        or "fuel_consumed" in colname
        or "net_generation" in colname
        or "fuel_mmbtu_per_unit" in colname
    ):
        pr_gen_fuel[colname] = pr_gen_fuel[colname].astype("float64")

In [4]:
# Handle EIA null values
pr_plant_frame = pr_plant_frame.replace(to_replace = ".", value = pd.NA)

# Convert data types (mmbtu/units to numeric, categories, booleans)
pr_plant_frame = handle_data_types(pr_plant_frame, categorical_cols = ["reporting_frequency_code", "data_maturity", "plant_state"])

In [5]:
#### monthly pivoting
# set up shared index
index_cols = ["plant_id_eia", "plant_name_eia", "report_year", "prime_mover_code", "energy_source_code", "fuel_unit"]

# Pivot variable columns
fuel_elec_mmbtu_melt = melt_monthly_vars(pr_gen_fuel, "fuel_consumed_for_electricity_mmbtu")
fuel_elec_units_melt = melt_monthly_vars(pr_gen_fuel, "fuel_consumed_for_electricity_units")
fuel_mmbtu_melt = melt_monthly_vars(pr_gen_fuel, "fuel_consumed_mmbtu")
fuel_units_melt = melt_monthly_vars(pr_gen_fuel, "fuel_consumed_units")
net_gen_melt = melt_monthly_vars(pr_gen_fuel, "net_generation_mwh")

# Combine all the pivoted DFs
pr_gen_fuel_melt = pd.concat(
    [fuel_elec_mmbtu_melt, fuel_elec_units_melt, fuel_mmbtu_melt, fuel_units_melt, net_gen_melt],
    axis="columns",
).reset_index()

In [6]:
## Create date from month and year
pr_gen_fuel_melt["date"] = pd.to_datetime(
    pr_gen_fuel_melt["month"] + pr_gen_fuel_melt["report_year"].astype(str),
    format="%B%Y",
)
## Drop old date columns
pr_gen_fuel_clean = pr_gen_fuel_melt.drop(columns = ["report_year", "month"])

In [ ]:
# Plant 62410 has two 2020 data entries but one is null
# Drop the bad row
pr_gen_fuel_final = pr_gen_fuel_clean.loc[
    ~((pr_gen_fuel_clean.plant_id_eia == 62410) 
    & (pr_gen_fuel_clean.date.dt.year == 2020)
    & (pr_gen_fuel_clean.fuel_consumed_for_electricity_mmbtu.isnull()))
]

# drop after 2025-03-01 (for now) as these values should not exist
pr_gen_fuel_final = pr_gen_fuel_final.loc[pr_gen_fuel_clean.date < pd.Timestamp("2025-03-01")]

In [ ]:
### output the data to Parquet files
pr_gen_fuel_final.to_parquet("../data/pr_gen_fuel_monthly.parquet")
pr_plant_frame.to_parquet("../data/pr_plant_frame.parquet")